Intro) A une époque ou la mahorité des réservations d"hôtels se font via des sites tiers tels que Booking.com, l'e-reputation est devenu un élément de préoccupation majeur pour les hôtels.Être capable d'analyser et d'extraire des insights à partir des commentaires en ligne (Note client moyenne, comparaison par rapport aux concurrents, mentions positivies/negatives...) est donc un atout pour les opérateurs d'hôtels.

Le but de ce projet est donc de réaliser une brève analyse NLP de 100k commentaires booking laissé sur les établissements du département des Pyrénnées-Atlantiques (64) entre 2019 et 2022.

Dans ce notebook, on lance un crawl via selenium pour recuperer les avis booking des établissements des Pyréennées Atlantiques via les urls récupérés dans le notebook 1- Crawl Urls Booking. Ces avis constitueront la base de données
utilisées pour l'analyse NLP. 

Un extrait des avis récupérés est disponible dans le fichier "Extrait_avis_booking.csv"


In [ ]:
import re
import time
import warnings
import concurrent.futures

from tqdm import tqdm
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from fake_useragent import UserAgent

from concurrent.futures import ProcessPoolExecutor
from concurrent.futures import Future
from concurrent.futures import ThreadPoolExecutor, as_completed

#Ignore deprectation & SSL certificate errors
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings('ignore', message='Unverified HTTPS request')

In [ ]:
#Get hotels urls and setting logs so we can keep track of which urls have been done 
try :
    with open("log.txt","r",encoding='utf-8') as log :
        done_urls = log.readlines()
        done_urls = [url.replace("\n","") for url in done_urls]
        log.close()
except Exception as e:
        pass

with open("booking_url.txt","r") as liste_hotels :
    liste_urls = liste_hotels.readlines()
    liste_urls = [url.replace("\n","") for url in liste_urls]
    liste_hotels.close()

try :
    liste_urls = [url for url in liste_urls if url not in done_urls]
except Exception as e:
    pass

In [ ]:
#Create an error file
with open('exception.txt',"w") as flog:
    print(datetime.datetime.now(),'\n')

In [ ]:
def loop(filename,url, hotel_name) :
    """
    Fonction prenant en paramètre un nom de fichier, un url d'un établissement sur 
    booking ainsi que le nom de l'établissement et qui va crawler l'ensemble 
    des commentaires listés sur la page de d'établissement puis les sauvegarder sur 
    le fichier précisé.
    
    Arguments:
        filename : String : Chemin d'accès du fichier de résultats
        
        url : String : url booking d'un établissement
        Par exemple : "https://www.booking.com/hotel/fr/logis-de-la-nivelle.fr.html"
        
        hotel_name : String : Nom booking d'un établissement
        Par exemple : "Logis Hotel de la Nivelle"
        
    Renvoit: Null
        Crawl et ecrit sur le fichier de résultat les avis clients 
        listés sur l'url de l'ébalissements
    """
    #Loop on one page of comments (Max of 10 comments per page)
    for i in range(1,11) :
        try :
            note = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/div[2]/div/div'.format(i))
            note = note.text
        except Exception as e:
            note = ""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)
        try :
            chambre = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[1]/div[2]/ul/li/a/div'.format(i))
            chambre = chambre.text
        except Exception as e:
            chambre =""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)
        try :
            durée = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[1]/ul[1]/li/div'.format(i))
            durée_sejour = durée.text.split(" ")[0]
            mois_sejour = durée.text.split(" ")[-2]
            année_sejour = durée.text.split(" ")[-1]
        except Exception as e:
            durée_sejour = ""
            mois_sejour = ""
            année_sejour = ""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)
        try :
            voyageur = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[1]/ul[2]/li/div'.format(i))
            voyageur = voyageur.text
        except Exception as e:
            voyageur = ""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)

        try :
            natio = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[1]/div[1]/div/div[2]/span[2]'.format(i))
            natio = natio.text
        except Exception as e:
            natio = ""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)

        try :
            #Avoid taking other stuff than the date of the comment
            date = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[2]/div[1]/span[2]'.format(i))
            date = date.text.replace("Commentaire envoyé le ","")
        except Exception as e:
            try :
                date = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[2]/div[1]/span'.format(i))
                date = date.text.replace("Commentaire envoyé le ","")
            except:
                date = ""
                with open('exception.txt',"a") as flog:
                    print('%r generated an exception: %s' % (url, e),file=flog)

        try :
            titre = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[2]/div[1]/div/div[1]/h3[1]'.format(i))
            titre = titre.text
        except Exception as e:
            titre = ""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)

        try :
            commentaire = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/ul/li[{}]/div/div[2]/div[2]/div[2]/div'.format(i))
            commentaire = commentaire.text.replace("\n","")
        except Exception as e:
            commentaire =""
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)

        #
        result=(url+'\t'+ str(hotel_name).replace('\t','').replace('\n','')+ '\t'+ str(note).replace('\t','').replace('\n','')+ '\t'+ str(chambre).replace('\t','').replace('\n','')+'\t'
                +str(durée_sejour).replace('\t','').replace('\n','') +'\t' +str(mois_sejour).replace('\t','').replace('\n','') +'\t' +str(année_sejour).replace('\t','').replace('\n','')
                +'\t' +str(voyageur).replace('\t','').replace('\n','')+'\t' +str(natio).replace('\t','').replace('\n','')+'\t' +str(date).replace('\t','').replace('\n','') +'\t' +str(titre).replace('\t','').replace('\n','')
                +'\t' +str(commentaire).replace('\t','').replace('\n',''))

        with open("{}.csv".format(filename),'a',encoding='utf-8') as file:
            print(result,file = file)

In [ ]:
#Main function
def crawl_hotel_comments(url) :
    """
    Fonction prenant en paramètre un url d'un établissement sur 
    booking et qui va crawler et stocker sur un fichier (1 csv par établissement)
    les avis clients associés.
    
    Cette fonction fait notamment appel à la fonction loop()
    définie au dessus
    
    Arguments:
        url : String : url booking d'un établissement
        Par exemple : "https://www.booking.com/hotel/fr/logis-de-la-nivelle.fr.html"
     
    Renvoit: Null
        Crawl et ecrit sur le fichier de résultat les avis clients 
        listés sur l'url de l'ébalissements
    """
    try :
        #Making sure all drivers don't start at the same time when multithreading
        time.sleep(random.uniform(1,5))

        timestamp = round(time.time()*1000000)

        filename_url = str(timestamp) + str(re.findall(r'fr\/(.+).fr.html',url)[0])

        #File to save results
        with open("{}.csv".format(filename_url),"w",encoding='utf-8') as file :
            print('Urls\tHotels\tNotes\tTypes_chambres\tDurées_sejours\tMois_sejours\tAnnees_sejours'+
                  '\tTypes_voyageurs\tNationalites\tDates_commentaires\tTitres_commentaires\tCommentaires',file=file)


        #Driver
        chrome_options = Options()
        ua = UserAgent()
        userAgent = ua.random
        chrome_options.add_argument(f"user-agent={userAgent}")
        path = r"DRIVER_PATH"
        chrome = webdriver.Chrome(executable_path = path, options = chrome_options)
        chrome.maximize_window()
        
        #Go to booking page
        chrome.get(url)
        time.sleep(2)
        
        #Nom hotel
        hotel_name = chrome.find_element_by_xpath('//*[@id="hp_hotel_name"]')
        hotel_name = hotel_name.text

        #Acceder commentaires
        try :
            btn_avis = chrome.find_element_by_xpath('//*[@id="show_reviews_tab"]')
            chrome.execute_script("arguments[0].scrollIntoView();", btn_avis)
            btn_avis.click()
            time.sleep(3)
        except Exception as e :
            with open('exception.txt',"a") as flog:
                print('%r generated an exception: %s' % (url, e),file=flog)

        #Loop on all pages of comments
        #First page
        loop(filename_url,url, hotel_name)
        last_page_not_reached = True
        #Other pages
        while last_page_not_reached :
            try :
                popup = chrome.find_element_by_xpath('//*[@id="onetrust-accept-btn-handler"]')
                chrome.execute_script("arguments[0].scrollIntoView();", popup)
                popup.click()
                time.sleep(3)
            except Exception as e :
                pass
            i = 0
            btn = False
            while True :
                if btn == False and i <3:
                    i += 1
                    try :
                        new_page = chrome.find_element_by_xpath('//*[@id="review_list_page_container"]/div[6]/div/div[1]/div/div[3]/a')
                        chrome.execute_script("arguments[0].scrollIntoView();", new_page)
                        new_page.click()
                        time.sleep(3)

                        btn = True
                        loop(filename_url,url, hotel_name)

                    except Exception as e:
                        with open('exception.txt',"a") as flog:
                            print('%r could not click next page btn: %s' % (url, e),file=flog)
                elif btn == True :
                    break
                elif i>=3:
                    with open('exception.txt',"a") as flog:
                        print('%r reached last page: ' % (url),file=flog)
                    last_page_not_reached = False
                    break
                    
        with open("log.txt",'a',encoding='utf-8') as file:
            print(url,file = file)
        chrome.quit()
    except Exception as e:
        with open('exception.txt',"a") as flog:
            print('%r generated an exception: %s' % (url, e),file=flog)
        chrome.quit()